In [562]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime

In [563]:
df = pd.read_csv("attacks.csv", engine="python")
df.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN


In [564]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

I check column by column with 'df["column"]' and 'df["column"].value_counts()' so that I can observe if the info is useful and if there are repeated categories or mixed formats. Results:

- All the columns have NaN values.
- Case Number: Probably unusefull. 
- Date: Mixed format.
- Year: Mixed format and wrong data.                  
- Type: Repeated categories with different names.                   
- Activity: Repeated categories with different names. 
- Name: Mixed data with sex. Not relevant.  I will ignore this column.  
- Sex: the index has a space on the right. A few repeated category because of the format. Missing values  I will ignore this column.
- Age: mixed format. Wrong data. Missing data. I will ignore this column.
- Fatal (Y/N): mixed format. Wrong data. 
- Time: mixed format.
- Species: the index has a space on the right. Repeated categories with different names.
- Investigator or Source: probably unusefull. 
- pdf:  probably unusefull. 
- href formula: Only usefull to get missing data.
- href: Only usefull to get missing data.
- Case Number.1: Probably unusefull.
- Case Number.2: Probably unusefull.
- original order: Probably unusefull.
- Unnamed: 22: Probably unusefull.
- Unnamed: 23: Probably unusefull.



In [565]:
df_clean = df.copy() # I make a copy, the clean one.

# I create list comprehension of the columns I don't need to drop it:
column_names = df_clean.columns
columns_unuseful = [column_names[i] for i in [0,8,9,10,15,16,17,18,19,20,21,22,23]]

df_clean.drop(columns_unuseful, inplace=True, axis=1)

In [566]:
#Now I make the column names upper case.
df_clean.columns = df_clean.columns.str.upper()

In [567]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE         6302 non-null   object 
 1   YEAR         6300 non-null   float64
 2   TYPE         6298 non-null   object 
 3   COUNTRY      6252 non-null   object 
 4   AREA         5847 non-null   object 
 5   LOCATION     5762 non-null   object 
 6   ACTIVITY     5758 non-null   object 
 7   INJURY       6274 non-null   object 
 8   FATAL (Y/N)  5763 non-null   object 
 9   TIME         2948 non-null   object 
 10  SPECIES      3464 non-null   object 
dtypes: float64(1), object(10)
memory usage: 2.2+ MB


In [568]:
# I get the sume of the NaN in my dataframe:
df_clean.isna().sum()

DATE           19421
YEAR           19423
TYPE           19425
COUNTRY        19471
AREA           19876
LOCATION       19961
ACTIVITY       19965
INJURY         19449
FATAL (Y/N)    19960
TIME           22775
SPECIES        22259
dtype: int64

In [569]:
#Then I drop NaN values keeping only the rows with at least 2 non-NA values:
df_clean = df_clean.dropna(thresh=2)
df_clean.isna().sum()

DATE              0
YEAR              2
TYPE              4
COUNTRY          50
AREA            455
LOCATION        540
ACTIVITY        544
INJURY           28
FATAL (Y/N)     539
TIME           3354
SPECIES        2838
dtype: int64

## YEAR

In [570]:
df_clean["YEAR"]

0       2018.0
1       2018.0
2       2018.0
3       2018.0
4       2018.0
         ...  
6297       0.0
6298       0.0
6299       0.0
6300       0.0
6301       0.0
Name: YEAR, Length: 6302, dtype: float64

In [571]:
df_clean[df_clean["YEAR"] == 0.0].head(3)
# We can leave out these 125 cases as they are not really significant, they are too old with a lot of missing date.

,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,INJURY,FATAL (Y/N),TIME,SPECIES
6177,Ca. 214 B.C.,0.0,Unprovoked,NaN,Ionian Sea,NaN,Ascending from a dive,"FATAL, shark/s bit him in two",Y,NaN,NaN
6178,Ca. 336.B.C..,0.0,Unprovoked,GREECE,Piraeus,In the haven of Cantharus,Washing his pig in preparation for a religious...,"FATAL, shark ""bit off all lower parts of him u...",Y,NaN,NaN
6179,493 B.C.,0.0,Sea Disaster,GREECE,Off Thessaly,NaN,Shipwrecked Persian Fleet,Herodotus tells of sharks attacking men in the...,Y,NaN,NaN


In [572]:
#After this evaluation I will keep only the relevant data which would be from 1950:
df_clean = df_clean[df_clean["YEAR"] >= 1950]
df_clean.head(3)

,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,INJURY,FATAL (Y/N),TIME,SPECIES
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,07h45,NaN



## DATE


In [573]:
df_clean["DATE"].value_counts()

1957                            11
1956                             8
1950                             7
1958                             7
28-Jul-1995                      5
                                ..
30-Mar-1971                      1
Mid Jul-1985 or mid Jul-1986     1
12-Aug-2014                      1
Reported      10-Dec-1994        1
20-Mar-1967                      1
Name: DATE, Length: 3817, dtype: int64

In [575]:
#df['Date'].str.extract(r'^(\d{4})', expand=False)
#print(set(df_clean["DATE"]))


In [576]:
df_clean["DATE"]=df_clean["DATE"].str.strip("Reported")
df_clean["DATE"]=df_clean["DATE"].str.strip(" Reported")
df_clean["DATE"]=df_clean["DATE"].str.strip(".Reported")
df_clean["DATE"]=df_clean["DATE"].str.strip("Late 1600s Reported")
df_clean["DATE"]=df_clean["DATE"].str.strip('Reported to have happened  "on the weekend"')


In [620]:
#df_clean["DATE_CLEAN"]=datetime.datetime.strptime(df_clean["DATE"],'%d-%b-%Y').strftime('%Y-%m-%d')
#m = df % 3 == 0
#df.where(m, -df)
#df_clean["Months"] = np.where(df_clean["Date"].str.contains(x), y, df_clean["Date"])


"""
def date_clean():
    for i in x:
        try:
            i.split("-")
        except


type(df_clean["DATE"])"""

'\ndef date_clean():\n    for i in x:\n        try:\n            i.split("-")\n        except\n\n\ntype(df_clean["DATE"])'

In [640]:
"""
for df_clean["DATE"] in df_clean:
    if df_clean["DATE"].str.contains("-"):
        df_clean["test"].append(i.split("-"))

print(df_clean)
"""

'\nfor df_clean["DATE"] in df_clean:\n    if df_clean["DATE"].str.contains("-"):\n        df_clean["test"].append(i.split("-"))\n\nprint(df_clean)\n'

In [623]:
print(set(df_clean["DATE"]))



{'20-Mar-1967', '29-Jun-1982', '-Jan-2', '25-Jun-2018', '29-Oct-1989', '20-Jun-2002', '2-Dec-1968', '26-Apr-2004', '8-Mar-195', '9-Oct-2005', '7-Mar-2015', '24-Oct-2003', '4-Sep-1999', '21-Nov-1998', '8-Feb-1985', '25-Dec-1964', '4-Feb-1963', '28-Dec-1959', '9-Dec-195', '8-Apr-199', '-Nov-2013', '5-Jun-19', '31-Dec-1957', '28-Nov-199', '-Oct-1963', '20-Feb-1972', '29-Jun-1989', '9-Jul-1994', '7-Jan-2004', 'May 1955', '8-Aug-1972', '9-Feb-2', 'Sep-1998', '26-Dec-1963', '24-Apr-1992', 'Aug-19', '24-May-1995', '9-Jul-2015', '31-Jul-2', '28-Jul-1954', '9-Jan-2015', '28-Aug-2', '21-Aug-1995', '4-Feb 1993', '9-Aug-1989', '27-Jul-19', '21-Mar-2007', '-May-2', '3-Jul-2015', '29-May-1999', '9-Aug-2', '28-Apr-2008', 'Jun-1968', '23-Jul-199', '25-Dec-1968', '-Oct-2008', '25-Aug-1967', '28-Jun-1964', '4-Dec-2', '-Apr-2017', '31-May-2002', '-Aug-1999', '9-Feb-1993', 'Feb-198', '20-Aug-2008', '23-Sep-1955', '9-Dec-198', '25-Jan-1959', '27-Dec-1958', '99', '-Jul-2003', '24-May-195', '3-Sep-1987', '2-

In [629]:
df_clean["MONTH"]

0           Jun
1           Jun
2           Jun
3           Jun
4           Jun
         ...   
4493     Summer
4494         95
4495         95
4496    Ca. 195
4497    Ca. 195
Name: MONTH, Length: 4497, dtype: object

In [579]:
#data['result'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

#df[Date_clean] = pd.to_datetime(df_clean["Date"])

## MONTH

I create a new columns only with months

In [580]:
#I try to create a new column with months. But it didn't work...
df_clean["MONTH"] = np.where(df_clean["DATE"].str.contains("Jan"), "January", df_clean["DATE"])
def months_column(x,y):
    df_clean["MONTH"] = np.where(df_clean["DATE"].str.contains(x), y, df_clean["MONTH"])
months_column("Jan","January")
months_column("Feb", "February")
months_column("Mar", "March")
months_column("Apr", "April")
months_column("May","May")
months_column("Jun","Jun")
months_column("Jul","July")
months_column("Aug","Augost")
months_column("Sep","September")
months_column("Oct","October")
months_column("Nov","November")
months_column("Dec","December")

df_clean['MONTH']=np.where(df_clean['MONTH'].str.contains("Ap"), "April", df_clean['MONTH'])
df_clean['MONTH']=np.where(df_clean['MONTH'].str.contains("Summer"), "Summer", df_clean['MONTH'])
df_clean['MONTH']=np.where(df_clean['MONTH'].str.contains("summer"), "Summer", df_clean['MONTH'])
df_clean['MONTH']=np.where(df_clean['MONTH'].str.contains("Fall"), "Fall", df_clean['MONTH'])
df_clean['MONTH']=np.where(df_clean['MONTH'].str.contains("Winter"), "Winter", df_clean['MONTH'])


In [581]:
print (df_clean['MONTH'].unique())

['Jun' 'May' 'April' 'March' 'February' 'January' 'December' 'November'
 'October' 'September' 'Augost' 'July' '2017.06.05' '2014' '2008.01.3'
 'Fall' 'Summer' '2004' '2' '999' '998' '99' '995'
 'incident of 1994 in Hong Kong' '994' '993' '992' '989' '988' '987' '98'
 '985' '984' 'Ca. 1983' '983' '982' '2-30-198' '979' '978' '97' '975'
 '974' '973' '972' 'Ca. 197' 'Winter' '968' '965' 'Early 1965' 'Ca. 1965'
 'Early 1963' '963' '962' 'Ca. 1962' '9' '960-19' 'Ca. 19' '21764' '959'
 '958-1959' '958' 'Circa 1958' '957' '95' '955' 'Ca. 1955' '9955' '954'
 '954 (same day as  1954.00.00.f)' '953' '952-1954' '952' '\n1951.12.15'
 '950.07.19' '950 - 195' 'Ca. 195']


## TYPE

In [582]:
df_clean["TYPE"]

0          Boating
1       Unprovoked
2          Invalid
3       Unprovoked
4         Provoked
           ...    
4493    Unprovoked
4494    Unprovoked
4495    Unprovoked
4496    Unprovoked
4497    Unprovoked
Name: TYPE, Length: 4497, dtype: object

In [583]:
print (df_clean['TYPE'].unique())

['Boating' 'Unprovoked' 'Invalid' 'Provoked' 'Questionable' 'Sea Disaster'
 nan 'Boat' 'Boatomg']


In [584]:
df_clean['TYPE']=df_clean['TYPE'].fillna("UNKNOWN")

df_clean['TYPE']=np.where(df_clean['TYPE'].str.startswith('Boa'), 
                            "Boating", 
                            df_clean['TYPE'])

In [585]:
print (df_clean['TYPE'].unique())

['Boating' 'Unprovoked' 'Invalid' 'Provoked' 'Questionable' 'Sea Disaster'
 'UNKNOWN']


## ACTIVITY

In [586]:
df_clean["ACTIVITY"]

0                                                Paddling
1                                                Standing
2                                                 Surfing
3                                                 Surfing
4                                             Free diving
                              ...                        
4493                                             Swimming
4494                                     Diving for coins
4495    Spearfishing, but walking carrying fish on end...
4496              Helmet diving, collecting trochus shell
4497                                                  NaN
Name: ACTIVITY, Length: 4497, dtype: object

In [587]:
df_clean['ACTIVITY'] = df_clean['ACTIVITY'].str.lower() 

In [588]:
df_clean['ACTIVITY_TYPE']=df_clean['ACTIVITY'].fillna("UNKNOWN")
df_clean['ACTIVITY_TYPE']=np.where(df_clean['ACTIVITY'].str.contains("surf"), "Surfing", df_clean['ACTIVITY'])
df_clean['ACTIVITY_TYPE']=np.where(df_clean['ACTIVITY'].str.contains("div"), "Diving", df_clean['ACTIVITY_TYPE'])
df_clean['ACTIVITY_TYPE']=np.where(df_clean['ACTIVITY'].str.contains("swim"), "Swiming", df_clean['ACTIVITY_TYPE'])
df_clean['ACTIVITY_TYPE']=np.where(df_clean['ACTIVITY'].str.contains("fish"), "Fishing", df_clean['ACTIVITY_TYPE'])



In [589]:
#print (df_clean['ACTIVITY_TYPE'].unique())

## COUNTRY

In [590]:
print (df_clean['COUNTRY'].unique())

['USA' 'AUSTRALIA' 'MEXICO' 'BRAZIL' 'ENGLAND' 'SOUTH AFRICA' 'THAILAND'
 'COSTA RICA' 'MALDIVES' 'BAHAMAS' 'NEW CALEDONIA' 'ECUADOR' 'MALAYSIA'
 'LIBYA' nan 'CUBA' 'MAURITIUS' 'NEW ZEALAND' 'SPAIN' 'SAMOA'
 'SOLOMON ISLANDS' 'JAPAN' 'EGYPT' 'ST HELENA, British overseas territory'
 'COMOROS' 'REUNION' 'FRENCH POLYNESIA' 'UNITED KINGDOM'
 'UNITED ARAB EMIRATES' 'PHILIPPINES' 'INDONESIA' 'CHINA' 'COLUMBIA'
 'CAPE VERDE' 'Fiji' 'DOMINICAN REPUBLIC' 'CAYMAN ISLANDS' 'ARUBA'
 'MOZAMBIQUE' 'FIJI' 'PUERTO RICO' 'ITALY' 'ATLANTIC OCEAN' 'GREECE'
 'ST. MARTIN' 'FRANCE' 'PAPUA NEW GUINEA' 'TRINIDAD & TOBAGO' 'KIRIBATI'
 'ISRAEL' 'DIEGO GARCIA' 'TAIWAN' 'JAMAICA' 'PALESTINIAN TERRITORIES'
 'GUAM' 'SEYCHELLES' 'BELIZE' 'NIGERIA' 'TONGA' 'SCOTLAND' 'CANADA'
 'CROATIA' 'SAUDI ARABIA' 'CHILE' 'ANTIGUA' 'KENYA' 'RUSSIA'
 'TURKS & CAICOS' 'UNITED ARAB EMIRATES (UAE)' 'AZORES' 'SOUTH KOREA'
 'MALTA' 'VIETNAM' 'MADAGASCAR' 'PANAMA' 'SOMALIA' 'NEVIS'
 'BRITISH VIRGIN ISLANDS' 'NORWAY' 'SENEGAL' 'YEMEN' 'G

In [591]:
df_clean['COUNTRY'] = df_clean['COUNTRY'].str.upper() 

In [592]:
print (df_clean['COUNTRY'].unique())

['USA' 'AUSTRALIA' 'MEXICO' 'BRAZIL' 'ENGLAND' 'SOUTH AFRICA' 'THAILAND'
 'COSTA RICA' 'MALDIVES' 'BAHAMAS' 'NEW CALEDONIA' 'ECUADOR' 'MALAYSIA'
 'LIBYA' nan 'CUBA' 'MAURITIUS' 'NEW ZEALAND' 'SPAIN' 'SAMOA'
 'SOLOMON ISLANDS' 'JAPAN' 'EGYPT' 'ST HELENA, BRITISH OVERSEAS TERRITORY'
 'COMOROS' 'REUNION' 'FRENCH POLYNESIA' 'UNITED KINGDOM'
 'UNITED ARAB EMIRATES' 'PHILIPPINES' 'INDONESIA' 'CHINA' 'COLUMBIA'
 'CAPE VERDE' 'FIJI' 'DOMINICAN REPUBLIC' 'CAYMAN ISLANDS' 'ARUBA'
 'MOZAMBIQUE' 'PUERTO RICO' 'ITALY' 'ATLANTIC OCEAN' 'GREECE' 'ST. MARTIN'
 'FRANCE' 'PAPUA NEW GUINEA' 'TRINIDAD & TOBAGO' 'KIRIBATI' 'ISRAEL'
 'DIEGO GARCIA' 'TAIWAN' 'JAMAICA' 'PALESTINIAN TERRITORIES' 'GUAM'
 'SEYCHELLES' 'BELIZE' 'NIGERIA' 'TONGA' 'SCOTLAND' 'CANADA' 'CROATIA'
 'SAUDI ARABIA' 'CHILE' 'ANTIGUA' 'KENYA' 'RUSSIA' 'TURKS & CAICOS'
 'UNITED ARAB EMIRATES (UAE)' 'AZORES' 'SOUTH KOREA' 'MALTA' 'VIETNAM'
 'MADAGASCAR' 'PANAMA' 'SOMALIA' 'NEVIS' 'BRITISH VIRGIN ISLANDS' 'NORWAY'
 'SENEGAL' 'YEMEN' 'GULF OF 

In [593]:
#Irealize that this methode change the NaN elements, so first of all I have to re-fill them.
df_clean['COUNTRY']=df_clean['COUNTRY'].fillna("UNKNOWN")
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("EMIRATES"), "UNITED ARAB EMIRATES", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("ENGLAND"), "UNITED KINGDOM", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("ST. MAARTIN"), "ST. MARTIN", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("BRITISH ISLES"), "UNITED KINGDOM", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("SUDAN?"), "SUDAN", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("NEW GUINEA"), "PAPUA NEW GUINEA", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("RED SEA"), "RED SEA", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("GRAND CAYMAN"), "CAYMAN ISLANDS", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("FEDERATED STATES OF MICRONESIA"), "MICRONESIA", df_clean['COUNTRY'])
df_clean['COUNTRY']=np.where(df_clean['COUNTRY'].str.contains("PACIFIC OCEAN "), "PACIFIC OCEAN", df_clean['COUNTRY'])


In [594]:
print (df_clean['COUNTRY'].unique())

['USA' 'AUSTRALIA' 'MEXICO' 'BRAZIL' 'UNITED KINGDOM' 'SOUTH AFRICA'
 'THAILAND' 'COSTA RICA' 'MALDIVES' 'BAHAMAS' 'NEW CALEDONIA' 'ECUADOR'
 'MALAYSIA' 'LIBYA' 'UNKNOWN' 'CUBA' 'MAURITIUS' 'NEW ZEALAND' 'SPAIN'
 'SAMOA' 'SOLOMON ISLANDS' 'JAPAN' 'EGYPT'
 'ST HELENA, BRITISH OVERSEAS TERRITORY' 'COMOROS' 'REUNION'
 'FRENCH POLYNESIA' 'UNITED ARAB EMIRATES' 'PHILIPPINES' 'INDONESIA'
 'CHINA' 'COLUMBIA' 'CAPE VERDE' 'FIJI' 'DOMINICAN REPUBLIC'
 'CAYMAN ISLANDS' 'ARUBA' 'MOZAMBIQUE' 'PUERTO RICO' 'ITALY'
 'ATLANTIC OCEAN' 'GREECE' 'ST. MARTIN' 'FRANCE' 'PAPUA NEW GUINEA'
 'TRINIDAD & TOBAGO' 'KIRIBATI' 'ISRAEL' 'DIEGO GARCIA' 'TAIWAN' 'JAMAICA'
 'PALESTINIAN TERRITORIES' 'GUAM' 'SEYCHELLES' 'BELIZE' 'NIGERIA' 'TONGA'
 'SCOTLAND' 'CANADA' 'CROATIA' 'SAUDI ARABIA' 'CHILE' 'ANTIGUA' 'KENYA'
 'RUSSIA' 'TURKS & CAICOS' 'AZORES' 'SOUTH KOREA' 'MALTA' 'VIETNAM'
 'MADAGASCAR' 'PANAMA' 'SOMALIA' 'NEVIS' 'BRITISH VIRGIN ISLANDS' 'NORWAY'
 'SENEGAL' 'YEMEN' 'GULF OF ADEN' 'SIERRA LEONE' 'LIBERIA' 'V

## AREA

In [595]:
#print(df_clean['AREA'].unique())

## FATAL (Y/N)

In [596]:
print(df_clean['FATAL (Y/N)'].unique())

['N' 'Y' nan 'M' 'UNKNOWN' '2017' ' N']


In [597]:
df_clean['FATAL (Y/N)']=df_clean['FATAL (Y/N)'].fillna("UNKNOWN")
df_clean['FATAL (Y/N)']=np.where(df_clean['FATAL (Y/N)'].str.contains("M"), "N", df_clean['FATAL (Y/N)'])
df_clean['FATAL (Y/N)']=np.where(df_clean['FATAL (Y/N)'].str.contains(" N"), "N", df_clean['FATAL (Y/N)'])
df_clean['FATAL (Y/N)']=np.where(df_clean['FATAL (Y/N)'].str.contains("2017"), "N", df_clean['FATAL (Y/N)'])


In [598]:
print(df_clean['FATAL (Y/N)'].unique())

['N' 'Y' 'UNKNOWN']


In [599]:
df_clean.isna().sum()

DATE                0
YEAR                0
TYPE                0
COUNTRY             0
AREA              217
LOCATION          272
ACTIVITY          320
INJURY             15
FATAL (Y/N)         0
TIME             1901
SPECIES          1602
MONTH               0
ACTIVITY_TYPE       0
dtype: int64

## SPECIES

In [602]:
print(df_clean['SPECIES '].unique())

['White shark' nan '2 m shark' ...
 "Tiger shark, 3.7 m to 4.3 m  [12' to 14']"
 'Alleged to involve a white shark "with little yellow eyes"'
 "2.7 m [9'] shark with black-tipped pectoral fins"]


In [607]:
df_clean['SPECIES'] = df_clean['SPECIES '].fillna("UNKNOWN")
df_clean['SPECIES'] = df_clean['SPECIES'].str.lower() 

In [608]:
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("white"), "white shark", df_clean['SPECIES'])
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("tiger"), "tiger shark", df_clean['SPECIES'])
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("bull"), "bull shark", df_clean['SPECIES'])
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("blacktip"), "blacktip shark", df_clean['SPECIES'])
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("lemon"), "lemon shark", df_clean['SPECIES'])
df_clean['SPECIES']=np.where(df_clean['SPECIES'].str.contains("grey"), "grey shark", df_clean['SPECIES'])

In [609]:
print(df_clean['SPECIES '].unique())

['White shark' nan '2 m shark' ...
 "Tiger shark, 3.7 m to 4.3 m  [12' to 14']"
 'Alleged to involve a white shark "with little yellow eyes"'
 "2.7 m [9'] shark with black-tipped pectoral fins"]


In [613]:
df_clean['SPECIES'] = df_clean['SPECIES'].fillna("UNKNOWN")
df_clean.isna().sum()
#What am I doing wrong?

DATE                0
YEAR                0
TYPE                0
COUNTRY             0
AREA              217
LOCATION          272
ACTIVITY          320
INJURY             15
FATAL (Y/N)         0
TIME             1901
SPECIES          1602
MONTH               0
ACTIVITY_TYPE       0
SPECIES             0
dtype: int64